In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report)
import os
from pckgs.majority import MajorityClassifier

DATA_DIR = "../data/processed"


In [4]:
if not os.path.exists(f"{DATA_DIR}/train_unbalanced.csv"):
    raise FileNotFoundError("Файл с данными не найден, проверьте что данные скачаны и выполнена их предобработка")

# Загрузка данных
df_train = pd.read_csv(f"{DATA_DIR}/train_balanced.csv")
X_train = df_train.drop('Churn', axis=1)
y_train = df_train['Churn']

df_train_unbalansed = pd.read_csv(f"{DATA_DIR}/train_unbalanced.csv")
X_train_unbalansed = df_train_unbalansed.drop('Churn', axis=1)
y_train_unbalansed = df_train_unbalansed['Churn']

df_test = pd.read_csv(f"{DATA_DIR}/test.csv")
X_test = df_test.drop('Churn', axis=1)
y_test = df_test['Churn']



In [5]:

majority_model = MajorityClassifier()
majority_model.fit(X_train_unbalansed, y_train_unbalansed)

y_pred = majority_model.predict(X_test)
y_proba = majority_model.predict_proba(X_test)

print("=" * 60)
print("БЕЙСЛАЙН: Majority class")
print("=" * 60)

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'roc_auc': roc_auc_score(y_test, y_proba[:, 1]),
    'f1_score': f1_score(y_test, y_pred, zero_division=0),
    'precision': precision_score(y_test, y_pred, zero_division=0),
    'recall': recall_score(y_test, y_pred, zero_division=0)
}

for metric, value in metrics.items():
    print(f"  {metric:12}: {value:.3f}")

[MajorityClassifier] Обучение завершено:
  Всего образцов: 5625
  Самый частый класс: 0 (73.4%, 4130 образцов)
  Распределение классов: {0: 0.7342222222222222, 1: 0.2657777777777778}
БЕЙСЛАЙН: Majority class
  accuracy    : 0.734
  roc_auc     : 0.500
  f1_score    : 0.000
  precision   : 0.000
  recall      : 0.000


In [6]:

lr_model = LogisticRegression(random_state=42, max_iter=100)
lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_test)
y_proba = lr_model.predict_proba(X_test)[:, 1]

print("=" * 60)
print("БЕЙСЛАЙН: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ")
print("=" * 60)

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'roc_auc': roc_auc_score(y_test, y_proba),
    'f1_score': f1_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred),
    'recall': recall_score(y_test, y_pred)
}

for metric, value in metrics.items():
    print(f"  {metric:12}: {value:.3f}")

print(f"Потребовалось итераций: {lr_model.n_iter_[0]}")

БЕЙСЛАЙН: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ
  accuracy    : 0.726
  roc_auc     : 0.828
  f1_score    : 0.593
  precision   : 0.490
  recall      : 0.749
Потребовалось итераций: 47


### Расшифровка метрик

1. Accuracy (Точность) - это значит что модель правильно предсказывает хх% всех клиентов
2. ROC-AUC - оценивает качество бинарного классификатора, измеряя площадь под ROC-кривой, она показывает, насколько хорошо модель различает два класса, варьируясь от 0 до 1
3. F1-Score - представляет собой гармоническое среднее между двумя другими метриками: Точностью (Precision) и Полнотой (Recall), помогая найти баланс между ними и особенно полезная для оценки моделей на несбалансированных данных. (Гармоническое среднее: Особый вид среднего, который штрафует за сильные перекосы. Если Precision или Recall близки к нулю, то и F1-Score будет близок к нулю, даже если другая метрика высока)
4. Precision  - это значит что из всех клиентов, которых модель предсказала как "уйдёт", ХХ% действительно уходят. Если модель говорит "клиент уйдёт", мы можем доверять этому предсказанию в хх% случаев
5. Recall (полнота) — это показатель эффективности модели машинного обучения, который измеряет, какую долю реально положительных случаев модель смогла правильно идентифицировать из всех имеющихся.


### Результаты 

Majority Class: Плохо (но так и должно быть)  
- Accuracy 0.734 - просто угадывает самый частый класс
- ROC-AUC 0.500 = случайное угадывание
- F1/Precision/Recall = 0 - вообще не находит ушедших  

Вывод: Это "нулевая" точка, которую должна преодолеть любая ML-модель


Логистическая регрессия:  Хорошо для бейслайна
- Accuracy 80.5% - уже учится, а не угадывает
- ROC-AUC 83.6% 
- F1 0.610 - разумный баланс точности и полноты
- Precision 0.650 - если модель говорит "уйдёт", в 65% случаев права
- Recall 0.575 - находит 57.5% ушедших (есть куда расти)